i want to run my py code multiple times, with different inputs & all at the same time with one exe.
My informations are all saved in one csv, & my problem is i dont know how i can run my script multiple times at the same time, with different inputs. My code is still the same but i need some other inputs in it from the csv.
So as example: my csv has 20 rows so my code will run 20 times, but in every row is another argument what i need.
if anyone know how this works, would be glad.

In [ ]:
import pandas as pd
data = pd.read_csv("your_csv.csv")

import random
x = random.randint(0, 20)
print(data[:, x])

In [23]:
step = 0
with open('step.txt', mode='r+', encoding='utf-8') as f:
    line = f.read()
    if line is not None:
        step = int(line)
    if step >= 20 or step < 0:
        step = 0

    data = data[:, step]
    # your logic to handle current line
    step = step + 1
    f.write(str(step))

ValueError: invalid literal for int() with base 10: ''

In [27]:
def main():
    while True:
        input_line = input("How many lines to display? ")
        if input_line.isnumeric():
            for i in range(int(input_line)):
                print("*" * i)
        elif input_line == '':
            break
        else:
            print("wrong input")
main()


*
**
***
****

*
**
***
****


ValueError: invalid literal for int() with base 10: ''

In [28]:
from random import randint
from dataclasses import dataclass
from typing import BinaryIO, ClassVar, Union

# A simple unbalanced tree node with an add method
@dataclass
class Node:
    # write to file as 2-byte integers
    int_size: ClassVar[int] = 2
    value: int = None
    address: int = None
    left: Union['Node', int] = None
    right: Union['Node', int] = None

    def add(self, value):
        p = self
        while True:
            if value > p.value:
                if p.right is None:
                    p.right = Node(value=value)
                    break
                else:
                    p = p.right
            elif value < p.value:
                if p.left is None:
                    p.left = Node(value=value)
                    break
                else:
                    p = p.left
            else:
                # value already in tree
                break

    @classmethod
    def _write_int(cls, f: BinaryIO, i: int):
        f.write(i.to_bytes(length=cls.int_size, byteorder='big'))

    @classmethod
    def _write_node(cls, f: BinaryIO, node: 'Node'):
        f.write(node.value.to_bytes(length=cls.int_size, byteorder='big'))
        f.write((node.left.address if node.left is not None else 0).to_bytes(length=cls.int_size, byteorder='big'))
        f.write((node.right.address if node.right is not None else 0).to_bytes(length=cls.int_size, byteorder='big'))

    @classmethod
    def _read_int(cls, f: BinaryIO):
        return int.from_bytes(f.read(cls.int_size), byteorder='big')

    @classmethod
    def _read_node(cls, f: BinaryIO, node: 'Node'):
        node.value = int.from_bytes(f.read(cls.int_size), byteorder='big')
        node.left = int.from_bytes(f.read(cls.int_size), byteorder='big')
        node.right = int.from_bytes(f.read(cls.int_size), byteorder='big')

    def save(self, f: BinaryIO):
        # seek the start of the file and write a dummy root record
        f.seek(0)
        self._write_int(f, self.value)
        self._write_int(f, 0)
        self._write_int(f, 0)

        stack = []
        p = self
        while True:
            if p.left is not None and p.left.address is None:
                stack.append(p)
                p = p.left
            elif p.right is not None and p.right.address is None:
                stack.append(p)
                p = p.right
            else:
                p.address = f.tell()
                self._write_node(f, p)
                if stack:
                    p = stack.pop()
                else:
                    break

        # rewrite root node with correct location of left and right nodes
        f.seek(0)
        self._write_node(f, self)

    def load(self, f: BinaryIO):
        f.seek(0)
        self.address = 0

        def load_side(side: str):
            # this function avoids repeating this code twice for .left and .right,
            # working around Python not having pointers, otherwise you'd just point at the attributes
            nonlocal f, p, stack
            if not isinstance(getattr(p, side), int):
                return False
            if not getattr(p, side):
                setattr(p, side, None)
            else:
                f.seek(getattr(p, side))
                setattr(p, side, Node())
                stack.append(p)
                p = getattr(p, side)
                self._read_node(f, p)
            return True

        stack = []
        p = self
        self._read_node(f, p)
        while True:
            if not load_side('left') and not load_side('right'):
                if stack:
                    p = stack.pop()
                else:
                    break

    def print(self):
        stack = []
        p = self
        left_done = False
        right_done = False
        while True:
            if not left_done and p.left is not None:
                stack.append((p, True, False))
                p, left_done, right_done = p.left, False, False
            elif not right_done and p.right is not None:
                stack.append((p, True, True))
                p, left_done, right_done = p.right, False, False
            else:
                print(f'value: {p.value}, '
                      f'left: {p.left.value if p.left else "none"}, '
                      f'right: {p.right.value if p.right else "none"}')
                if stack:
                    p, left_done, right_done = stack.pop()
                else:
                    break


values = [randint(0, 100) for __ in range(10)]
root = Node(values[0])
for v in values[1:]:
    root.add(v)
print('\nTree after construction:')
root.print()

# save the tree
with open('tree.bin', 'wb') as f:
    root.save(f)

# start a new tree, initialise with single 0 because it needs one
root = Node()
# read the tree
with open('tree.bin', 'rb') as f:
    root.load(f)
print('\nResult after load:')
root.print()


Tree after construction:
value: 27, left: none, right: none
value: 37, left: 27, right: none
value: 82, left: none, right: none
value: 86, left: 82, right: none
value: 96, left: none, right: none
value: 91, left: none, right: 96
value: 90, left: 86, right: 91
value: 80, left: 37, right: 90

Result after load:
value: 27, left: none, right: none
value: 37, left: 27, right: none
value: 82, left: none, right: none
value: 86, left: 82, right: none
value: 96, left: none, right: none
value: 91, left: none, right: 96
value: 90, left: 86, right: 91
value: 80, left: 37, right: 90


In [36]:
def random_int_by_probas_sum(probas_cum, r):
    """
    @param probas_cum cumulation probablities of the 10 random numbers.
    @param r random number
    return the value of r-percentage. for example r=0.3, it will return 3, r=0.8, it will return 8
    """
    for index in range(len(probas_cum)):
        if r <= probas_cum[index]:
            return index

import numpy as np
import pandas as pd
import random

nums = np.arange(0, 10, 1)
#
probas = [0.05, 0.15, 0.07, 0.13, 0.09, 0.11, 0.08, 0.12, 0.04, 0.16]
probas_cum = [0.05, 0.2, 0.27, 0.4, 0.49, 0.6, 0.68, 0.8, 0.84, 1]
random_count = 1000000
sums = []

for _ in range(random_count):
    sum_step = 0
    for i in range(10):
        r = random.random()
        sum_step += random_int_by_probas_sum(probas_cum, r)
    sums.append(sum_step)

sums = pd.Series(sums)
sums.value_counts()


47    42919
46    42721
45    42625
48    42595
49    41979
      ...  
10        2
86        1
8         1
9         1
87        1
Length: 79, dtype: int64

In [38]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import numpy as np

data = np.ones((256,256))
fig = plt.figure()
ax = fig.add_subplot(111)
temp = np.transpose(data)
plt.title('title')
im = ax.imshow((temp), aspect='auto')
fig.colorbar(im)
plt.axes()

data2 = np.ones((256,256))
fig = plt.figure()
ax = fig.add_subplot(111)
temp = np.transpose(data2)
plt.title('title')
im = ax.imshow((temp), aspect='auto')
fig.colorbar(im)
plt.axes()
plt.show()

<ipython-input-38-cbd639ff1f9a>:13: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.axes()
<ipython-input-38-cbd639ff1f9a>:22: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.axes()


KeyboardInterrupt: 